# Import

In [ ]:
import pandas as pd
import numpy as np

# Preparation 

In [ ]:
dfPosts = pd.read_csv('../tweets.csv',low_memory=False)

# Retweet classification

In [ ]:
dfProvaxRt = pd.read_csv('retweet_class/tweets_provax.csv',low_memory=False)
dfNovaxRt = pd.read_csv('retweet_class/tweets_novax.csv',low_memory=False)
dfDoubledRt = pd.read_csv('retweet_class/tweets_doubled_classified.csv',low_memory=False,usecols=['user_id'])

## Redirect of doubled classified suspect

In [ ]:
doubled_class = dfDoubledRt['user_id'].unique()

#TODO with networks
dfNovaxRt = dfNovaxRt[dfNovaxRt['user_id'].isin(doubled_class) == False]
dfProvaxRt = dfProvaxRt[dfProvaxRt['user_id'].isin(doubled_class) == False]

# .CSV Creation 

### Novax

In [ ]:
dfNovax = pd.concat([dfNovaxRt])
dfNovax.drop_duplicates(['id'],inplace=True)
dfNovax

### High credibility

In [ ]:
dfProvax = pd.concat([dfProvaxRt])
dfProvax.drop_duplicates(['id'],inplace=True)
dfProvax

### Other user

In [ ]:
idNovax = dfNovax.groupby('id').first().index
dfOtherElem = dfPosts[dfPosts['id'].isin(idNovax) == False]

idHigh = dfProvax.groupby('id').first().index
dfOtherElem = dfOtherElem[dfOtherElem['id'].isin(idHigh) == False]

dfOtherElem

## Control if the dataframe are correctly created

In [ ]:
numPostControl = len(dfPosts) == len(dfProvax) + len(dfNovax) + len(dfOtherElem)
numPostControl

In [ ]:
numUserControl = len(dfPosts.groupby('user_screen_name').first().index) ==\
                                                        len(dfProvax.groupby('user_screen_name').first().index) \
                                                        + len(dfNovax.groupby('user_screen_name').first().index) \
                                                        + len(dfOtherElem.groupby('user_screen_name').first().index)
numUserControl

### Write dataframe to .csv

In [ ]:
if (numPostControl & numUserControl):
    dfOtherElem.to_csv('../tweets_notClass.csv', index=False)
    dfProvax.to_csv('../tweets_provax.csv', index=False)
    dfNovax.to_csv('../tweets_novax.csv', index=False)
else:
    print('The users are not correctly divided')